# #01 Geometry Creation
<i>Guide on how to create features (points, lines, surfaces and volumes)</i>
***

Connect on LUSAS and check if a model is open

In [ ]:
import math
import sys; sys.path.append('../') # Reference modules in parent directory
from LPI_22_0 import *
lusas = get_lusas_modeller()

if not lusas.existsDatabase():
    raise Exception("A model must be open before running this code")

## 1. Creating Points

In [ ]:
# The options and settings for creating all geometry in LUSAS is defined in the IFGeometryData object.
# To start any geometry creation, get the geometryData object and set all the defaults
geometry_data = lusas.geometryData().setAllDefaults()
# Here we are creating points by specifying the coordinates, we set the option in setLowerOrderGeometryType
geometry_data.setLowerOrderGeometryType("coordinates")
# Now we add the coordinates, first the origin
geometry_data.addCoords(0.0, 0.0, 0.0)
# And a second point, at X=1
geometry_data.addCoords(1.0, 0.0, 0.0)
# Now we instruct the database to the create the line. It will return an IFObject set containing the newly created points
object_set:'IFObjectSet' = lusas.database().createPoint(geometry_data)
# Get all points from the returned object set
points:list[IFPoint] = object_set.getObjects("Point")
# Loop through the points and print their ids and coordinates
for point in points:
    print(f"Point:{point.getID()} at coordinates ({point.getX()},{point.getY()},{point.getZ()}) ")

It is often useful to create a small helper function to wrap these settings as follows:

In [ ]:
def create_point(x:float, y:float, z:float) -> 'IFPoint':
    geometry_data = lusas.geometryData().setAllDefaults()
    geometry_data.setLowerOrderGeometryType("coordinates")
    geometry_data.addCoords(x, y, z)
    # Return the point object from the object set containing the newly created objects
    return win32.CastTo(lusas.database().createPoint(geometry_data).getObject("Point"), "IFPoint")

# Call the function to create a points
point1 = create_point(0.0, 0.0, 1.0)
point2 = create_point(1.0, 0.0, 1.0)

## 2. Creating Lines

### 2.1 Straight line By Coordinates

In [ ]:
# The options and settings for creating all geometry in LUSAS is defined in the IFGeometryData object.
# To start any geometry creation, get the geometryData object and set all the defaults
geometry_data = lusas.geometryData().setAllDefaults()
# Here we are creating a line by specifying the coordinates, we set the option in setLowerOrderGeometryType
geometry_data.setLowerOrderGeometryType("coordinates")
# We want a straight line
geometry_data.setCreateMethod("straight")
# Now we add the start and end coordinates
geometry_data.addCoords(0.0, 2.0, 0.0)
geometry_data.addCoords(5.0, 2.0, 0.0)
# Now we instruct the database to the create the line. It will return an IFObject set containing the newly created line
object_set:'IFObjectSet' = lusas.database().createLine(geometry_data)
# Get all points from the returned object set
lines:list[IFLine] = object_set.getObjects("Line")
# Loop through the lines and print their ids and coordinates
for line in lines:
    p1 = line.getStartPoint()
    p2 = line.getEndPoint()
    print(f"Line:{line.getID()} between points ({p1.getX()},{p1.getY()},{p1.getZ()}) to ({p2.getX()},{p2.getY()},{p2.getZ()}) ")

#### 2.2 Arc line by cooordinates

In [ ]:
# The options and settings for creating all geometry in LUSAS is defined in the IFGeometryData object.
# To start any geometry creation, get the geometryData object and set all the defaults
geometry_data = lusas.geometryData().setAllDefaults()
# Here we are creating a line by specifying the coordinates, we set the option in setLowerOrderGeometryType
geometry_data.setLowerOrderGeometryType("coordinates")
# We want an arc
geometry_data.setCreateMethod("arc")
# Specify how we are supplying the coordinates, i.e Start->Middle->End
geometry_data.setStartMiddleEnd()
# Now we add the start middle end coordinates
geometry_data.addCoords(0.0, 1.0, 0.0)
geometry_data.addCoords(2.5, 2.0, 0.0)
geometry_data.addCoords(5.0, 1.0, 0.0)

# Now we instruct the database to the create the line. It will return an IFObject set containing the newly created line
object_set:'IFObjectSet' = lusas.database().createLine(geometry_data)
# Get all points from the returned object set
lines:list[IFLine] = object_set.getObjects("Line")
# Loop through the lines and print their ids and coordinates
for line in lines:
    p1 = line.getStartPoint()
    p2 = line.getEndPoint()
    print(f"Line:{line.getID()}, length:{line.getLineLength()} between points ({p1.getX()},{p1.getY()},{p1.getZ()}) to ({p2.getX()},{p2.getY()},{p2.getZ()}) ")

### 2.3 Circle

In [ ]:
radius = 15
z = 3.5
# The options and settings for creating all geometry in LUSAS is defined in the IFGeometryData object.
# To start any geometry creation, get the geometryData object and set all the defaults
geometry_data = lusas.geometryData().setAllDefaults()
# Specify the line by coordinate input
geometry_data.setLowerOrderGeometryType("coordinates")
# Specify defining a circle
geometry_data.makeCircle()
# Specify the meaning of the following coordinates 
geometry_data.setStartEndCentre()
# Start point at X=radius
geometry_data.addCoords(radius, 0.0, z)
# Specify the plane (X/Y)
geometry_data.addCoords(0.0, 1.0, z)
# Centre of the circle at the origin
geometry_data.addCoords(0.0, 0.0, z)
# Create the line in the database with the settings specified.
object_set:'IFObjectSet' = lusas.database().createLine(geometry_data)


### 2.4 Create a Line from existing points

Existing points can also be used to create lines. This can be done adding the start and end points in an IFObjectSet. You can do so either by using each IFPoint object or by using each point's id:

In [ ]:
# Get id of second point
point2_id = point2.getID()
print(f"Point2 ID is {point2_id}")

# geometryData object contains all the settings to perform a geometry creation
geom_data = lusas.geometryData().setAllDefaults()        
# set the options for creating straight lines from points
geom_data.setCreateMethod("straight")
geom_data.setLowerOrderGeometryType("points")
# Create an object set to contain the points and use this set to create the line
obs = lusas.newObjectSet()

# Add existing points to the object set
obs.add(point1) # Add first point IFPoint object
obs.add("point", point2_id) # Add second point using the ID

# Create the line, get the line objects array from the returned object set and return the 1 and only line
new_line:IFLine = obs.createLine(geom_data).getObjects("Line")

Create a helper function that can be re-used later

In [ ]:
def create_line_from_points(p1:'IFPoint', p2:'IFPoint') -> 'IFLine':
     # geometryData object contains all the settings to perform a geometry creation
     geom_data = lusas.geometryData().setAllDefaults()         
     # set the options for creating straight lines from points
     geom_data.setCreateMethod("straight")        
     geom_data.setLowerOrderGeometryType("points")
     # Create an object set to contain the points and use this set to create the line
     obs = lusas.newObjectSet()                 
     obs.add(p1)
     obs.add(p2)
     # Create the line, get the line objects array from the returned object set and return the 1 and only line
     return win32.CastTo(obs.createLine(geom_data).getObject("Line"), "IFLine")

Create a circle of points and connect them with straight lines

In [ ]:
radius = 15
intersections = 10
# Store the list of points
points = []

for i in range(0, intersections):
    theta = i * math.radians(360/intersections)
    x = radius * math.cos(theta)
    y = radius * math.sin(theta)
    point = create_point(x, y, z)
    points.append(point)

# Create lines between each of the created points
for i in range(0, len(points)-1):
    create_line_from_points(points[i], points[i+1])
create_line_from_points(points[-1], points[0])

In [ ]:
radius = 15
intersections = 10
floor_height = 3.5
no_floors    = 3
z = 3.5 * 3

start_angle = 0.0

points = []
for i in range(0, intersections):
    theta = i * math.radians(360/intersections)
    x = radius * math.cos(theta + start_angle)
    y = radius * math.sin(theta + start_angle)
    point = create_point(x, y, 0.0)
    points.append(point)

for i in range(0, len(points)-1):
    create_line_from_points(points[i], points[i+1])
create_line_from_points(points[-1], points[0])

start_angle = math.radians(360/intersections)/2
points2 = []
for i in range(0, intersections):
    theta = i * math.radians(360/intersections)
    x = radius * math.cos(theta + start_angle)
    y = radius * math.sin(theta + start_angle)
    point = create_point(x, y, floor_height*no_floors)
    points2.append(point)

for i in range(0, len(points2)-1):
    create_line_from_points(points2[i], points2[i+1])
create_line_from_points(points2[-1], points2[0])


for i in range(0, len(points)):
    create_line_from_points(points[i], points2[i])

for i in range(0, len(points2)-1):
    create_line_from_points(points2[i], points[i+1])

## 3 Create Surfaces

### 3.1 Create a Surface

In [ ]:
# Create a geometryData object to contain all the settings for the geometry creation
geom_data = lusas.newGeometryData()
# set the options for creating geometries from coordinates
geom_data.setCreateMethod("coons")
geom_data.setLowerOrderGeometryType("coordinates")

# Specify surface coordinates
geom_data.addCoords(0, 0, -5)
geom_data.addCoords(1, 0, -5)
geom_data.addCoords(1, 1, -5)
geom_data.addCoords(0, 1, -5)

new_surface : IFSurface = lusas.db().createSurface(geom_data).getObjects("Surface")[0]

Create a helper function that can be re-used later

In [ ]:
def createSurfaceByCoordinates(x:list[float], y:list[float], z:list[float]) -> IFSurface:
    geom_data = lusas.newGeometryData()
    geom_data.setCreateMethod("coons")
    geom_data.setLowerOrderGeometryType("coordinates")
    for i in range(len(x)):
        geom_data.addCoords(x[i], y[i], z[i])
    surf : IFSurface = lusas.db().createSurface(geom_data).getObjects("Surface")[0]
    return surf

# Create a surface using the function
xs = [0, 1, 1, 0]
ys = [0, 0, 1, 1]
zs = [-4, -4, -4, -4]
surface1 = createSurfaceByCoordinates(xs, ys, zs)